# Problem 2: Reservoir Distribution with Valves

We are given 3 reservoirs connected in a mesh topology by bidirectional valves.  
Each reservoir has an initial amount of water and a target amount it must reach.

Water transfers happen by opening a valve between two reservoirs. When a valve is open, water flows until:

- The destination reservoir is **full**, OR  
- The source reservoir reaches the **safety threshold** (≥20% of its total capacity).  

At most **one valve** can be opened at a time. The total amount of water is conserved.

The goal is to determine the **sequence of valve operations** that transforms the initial distribution into the target distribution, while **minimizing the number of valve openings**.

---

## Input Format

- First line: `C1 C2 C3` → capacities of reservoirs  
- Second line: `I1 I2 I3` → initial water amounts  
- Third line: `T1 T2 T3` → target water amounts  

---

## Output Format

Print the sequence of valve operations in the format:

Open valve (X→Y)


using **BFS, DFS, and A\*** search (specify heuristics).  
Finally, print the number of valve operations used.

---

## Constraints

- $1 \leq C_i \leq 100$ (capacity of reservoir $i$)  
- $0 \leq I_i \leq C_i$ (initial water in reservoir $i$)  
- $0 \leq T_i \leq C_i$ (target water in reservoir $i$)  
- $I_1 + I_2 + I_3 = T_1 + T_2 + T_3$ (total water conserved)  
- $20\% \leq \text{water in reservoir} \leq 100\%$ at any time  



In [9]:
from collections import deque, namedtuple
import heapq
import math

# Helper Functions
def round_state(state):
    return tuple(round(x + 1e-9, 1) for x in state)

def apply_action(state, caps, i, j):
    state = list(state)
    src = state[i]
    dst = state[j]
    cap_src = caps[i]
    cap_dst = caps[j]
    safety_min_src = 0.2 * cap_src
    src_can_give = round(max(0.0, src - safety_min_src), 9)
    dst_can_accept = round(max(0.0, cap_dst - dst), 9)
    transfer = min(src_can_give, dst_can_accept)
    if transfer <= 1e-9:
        return None, 0.0
    state[i] = round(src - transfer, 9)
    state[j] = round(dst + transfer, 9)
    new_state = round_state(state)
    return new_state, round(transfer, 1)

def is_goal(state, target, tol=1e-6):
    return all(abs(state[i] - target[i]) < tol for i in range(3))

def actions_from_state(state, caps):
    acts = []
    for i in range(3):
        for j in range(3):
            if i == j:
                continue
            new_state, transfer = apply_action(state, caps, i, j)
            if new_state is not None:
                acts.append(((i, j), new_state, transfer))
    return acts

def reconstruct(parents, end_state):
    path = []
    cur = end_state
    while parents[cur][0] is not None:
        act = parents[cur][1]  # (i,j)
        path.append(act)
        cur = parents[cur][0]
    path.reverse()
    return path

# BFS
def bfs(start, target, caps):
    start = round_state(start)
    target = round_state(target)
    q = deque([start])
    parents = {start: (None, None)}
    while q:
        s = q.popleft()
        if is_goal(s, target):
            path = reconstruct(parents, s)
            return path, s
        for (i,j), new_s, transfer in actions_from_state(s, caps):
            if new_s not in parents:
                parents[new_s] = (s, (i,j))
                q.append(new_s)
    return None, None

# DFS (iterative)
def dfs(start, target, caps, max_nodes=20000):
    start = round_state(start)
    target = round_state(target)
    stack = [start]
    parents = {start: (None, None)}
    visited = set([start])
    nodes = 0
    while stack and nodes < max_nodes:
        s = stack.pop()
        nodes += 1
        if is_goal(s, target):
            return reconstruct(parents, s), s
        for (i,j), new_s, transfer in actions_from_state(s, caps):
            if new_s not in visited:
                visited.add(new_s)
                parents[new_s] = (s, (i,j))
                stack.append(new_s)
    return None, None

# A*
def astar(start, target, caps):
    start = round_state(start)
    target = round_state(target)

    def heuristic(state):
        # total "distance" of water between current and target
        diff = sum(abs(si - ti) for si, ti in zip(state, target))
        needed_transfer = diff / 2.0
        max_transfer = 0.8 * max(caps)
        return math.ceil(needed_transfer / max_transfer)

    open_heap = []
    gscore = {start: 0}
    parents = {start: (None, None)}
    fscore = {start: heuristic(start)}
    heapq.heappush(open_heap, (fscore[start], start))
    closed = set()

    while open_heap:
        _, s = heapq.heappop(open_heap)
        if s in closed:
            continue
        if is_goal(s, target):
            return reconstruct(parents, s), s
        closed.add(s)
        for (i,j), new_s, transfer in actions_from_state(s, caps):
            tentative_g = gscore[s] + 1  # each action costs 1 (minimize number of valve openings)
            if new_s in closed:
                continue
            if new_s not in gscore or tentative_g < gscore[new_s]:
                gscore[new_s] = tentative_g
                parents[new_s] = (s, (i,j))
                f = tentative_g + heuristic(new_s)
                heapq.heappush(open_heap, (f, new_s))
    return None, None

caps = (8.0, 5.0, 3.0)
start = (8.0, 0.0, 0.0)
target = (2.4, 5.0, 0.6)


b_path, b_end = bfs(start, target, caps)
print("BFS result:")
if b_path is not None:
    print("\n".join([f"Open valve ({a[0]+1}->{a[1]+1})" for a in b_path]))
    print("Number of valve operations =", len(b_path))
else:
    print("No solution found by BFS")

d_path, d_end = dfs(start, target, caps)
print("\nDFS result:")
if d_path is not None:
    print("\n".join([f"Open valve ({a[0]+1}->{a[1]+1})" for a in d_path]))
    print("Number of valve operations =", len(d_path))
else:
    print("No solution found by DFS")

a_path, a_end = astar(start, target, caps)
print("\nA* result (h=0):")
if a_path is not None:
    print("\n".join([f"Open valve ({a[0]+1}->{a[1]+1})" for a in a_path]))
    print("Number of valve operations =", len(a_path))
else:
    print("No solution found by A*")



BFS result:
Open valve (1->2)
Open valve (1->3)
Open valve (3->1)
Number of valve operations = 3

DFS result:
Open valve (1->3)
Open valve (3->2)
Open valve (1->2)
Number of valve operations = 3

A* result (h=0):
Open valve (1->2)
Open valve (1->3)
Open valve (3->1)
Number of valve operations = 3
